In [3]:
import skmob
import pandas as pd
import plotly.express as px
from tqdm import tqdm
import geocoder
import datetime

tqdm.pandas()

c:\users\askib\pycharmprojects\mobility\venv\lib\site-packages\tqdm\std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [9]:
path = "../../data/raw/KBR_Gadow_2019_bez_hasla.xlsx"

In [25]:
df = pd.read_excel(path, header=[0,1])

In [26]:
columns = list(df.columns)[199:]
columns.append(list(df.columns)[0])
# df = df[l]
df = df[columns]

In [27]:
df.columns = df.columns.droplevel(1)
df.rename(columns={df.columns[270]:'ID odpowiedzi'}, inplace=True)
df.columns

Index(['ID podróży (1)', 'Data rozpoczecia podróży (1)',
       'Czas trwania podróży [s] (1)', 'Środek transportu (1)',
       'Typ miejsca startu (1)', 'Typ miejsca zakończenia (1)',
       'Miejsce startu (1)', 'Numer regionu miejsca startu (1)',
       'Numer regionu miejsca zakończenia (1)', 'Miejsce zakończenia (1)',
       ...
       'Data rozpoczecia podróży (27)', 'Czas trwania podróży [s] (27)',
       'Środek transportu (27)', 'Typ miejsca startu (27)',
       'Typ miejsca zakończenia (27)', 'Miejsce startu (27)',
       'Numer regionu miejsca startu (27)',
       'Numer regionu miejsca zakończenia (27)', 'Miejsce zakończenia (27)',
       'ID odpowiedzi'],
      dtype='object', length=271)

In [13]:
new_columns = [
    'user_id',
    'travel_id',
    'start_date',
    'duration_time',
    'mean_of_transport',
    'start_place_type',
    'finish_place_type',
    'start_place',
    'start_region',
    'residence_region',
    'finish_place'
]

travels_df = pd.DataFrame(columns=new_columns)

for i in tqdm(range(1, 28)):
    columns = [
        'ID odpowiedzi',
        f'ID podróży ({i})',
        f'Data rozpoczecia podróży ({i})',
        f'Czas trwania podróży [s] ({i})',
        f'Środek transportu ({i})',
        f'Typ miejsca startu ({i})',
        f'Typ miejsca zakończenia ({i})',
        f'Miejsce startu ({i})',
        f'Numer regionu miejsca startu ({i})',
        f'Numer regionu miejsca zakończenia ({i})',
        f'Miejsce zakończenia ({i})'
    ]

    sub_df = df[columns]
    sub_df = sub_df.dropna()
    sub_df.columns = new_columns

    travels_df = travels_df.append(sub_df)

100%|██████████| 27/27 [00:00<00:00, 56.71it/s]


In [14]:
travels_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 0 to 259
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            1277 non-null   object 
 1   travel_id          1277 non-null   object 
 2   start_date         1277 non-null   object 
 3   duration_time      1277 non-null   object 
 4   mean_of_transport  1277 non-null   object 
 5   start_place_type   1277 non-null   object 
 6   finish_place_type  1277 non-null   object 
 7   start_place        1277 non-null   object 
 8   start_region       1277 non-null   float64
 9   residence_region   1277 non-null   float64
 10  finish_place       1277 non-null   object 
dtypes: float64(2), object(9)
memory usage: 119.7+ KB


In [ ]:
tdf = pd.DataFrame(columns=['travel_id', 'lat', 'long', 'time', 'mean_of_transport'])

for index, row in tqdm(travels_df.iterrows(), total=travels_df.shape[0]):
    g_start = geocoder.osm(row['start_place'])
    g_finish = geocoder.osm(row["finish_place"])

    try:
        tdf = tdf.append({
            'travel_id': row['travel_id'],
            'lat': g_start.osm['x'],
            'long': g_start.osm['y'],
            'time': datetime.datetime.strptime(row["start_date"], '%Y-%m-%d %H:%M:%S UTC'),
            'mean_of_transport': row['mean_of_transport']
        }, ignore_index=True)

        tdf = tdf.append({
            'travel_id': row['travel_id'],
            'lat': g_finish.osm['x'],
            'long': g_finish.osm['y'],
            'time': datetime.datetime.strptime(row["start_date"], '%Y-%m-%d %H:%M:%S UTC')
 + datetime.timedelta(0,int(row["duration_time"])),
            'mean_of_transport': row['mean_of_transport']
        }, ignore_index=True)
    except Exception as e:
        print()
        print(e)
        print(g_start.osm)
        print(g_finish.osm)

  5%|▌         | 68/1277 [01:08<20:04,  1.00it/s]

In [30]:

tdf.to_csv("../data/interim/flow_matrix_gadow.csv")

In [18]:
df = pd.read_csv("../../data/processed/flow_matrix_gadow.csv")

In [20]:
df

,Unnamed: 0,travel_id,lat,long,time,mean_of_transport
0,0,798a9138-afb4-4599-a4a4-b953673a6586,16.963825,51.126405,2019-10-27 07:40:00,Pieszo
1,1,798a9138-afb4-4599-a4a4-b953673a6586,16.960701,51.127634,2019-10-27 07:48:00,Pieszo
2,2,5e3a9ce9-2285-41d5-a395-a483adbc45f6,16.957725,51.126554,2019-10-08 05:00:00,Samochód osobowy jako kierowca
3,3,5e3a9ce9-2285-41d5-a395-a483adbc45f6,16.986002,51.112351,2019-10-08 05:30:00,Samochód osobowy jako kierowca
4,4,9cd2247e-4c96-4d73-b81c-7ee752b099ae,16.969748,51.126558,2019-10-27 18:30:00,Samochód osobowy jako kierowca
...,...,...,...,...,...,...
2539,2539,21ac528a-30ea-4612-881e-ef74e0653fea,17.022057,51.111591,2019-11-15 21:35:00,Pieszo
2540,2540,21ac528a-30ea-4612-881e-ef74e0653fea,17.022057,51.111591,2019-11-15 21:30:00,Tramwaj
2541,2541,21ac528a-30ea-4612-881e-ef74e0653fea,16.952153,51.135690,2019-11-15 21:50:00,Tramwaj
2542,2542,21ac528a-30ea-4612-881e-ef74e0653fea,16.952153,51.135690,2019-11-15 21:30:00,Pieszo


In [24]:
travels_df["finish_place_type"].value_counts()

home                    316
work                    288
intermediate            251
other                   208
shop                    122
school_or_university     63
park                     21
office                    8
Name: finish_place_type, dtype: int64

In [23]:

pd.crosstab(travels_df["start_place_type"],travels_df["finish_place_type"] )

finish_place_type,home,intermediate,office,other,park,school_or_university,shop,work
start_place_type,,,,,,,,
home,2,104,3,116,15,43,44,180
intermediate,55,71,1,36,3,8,14,60
office,1,1,0,0,0,1,1,0
other,89,17,0,19,0,0,10,13
park,5,1,0,1,2,0,0,0
school_or_university,17,9,0,1,0,2,4,15
shop,66,6,0,8,1,0,9,4
work,81,42,4,27,0,9,40,16


In [34]:

columns = ['ID odpowiedzi']
for i in tqdm(range(1, 28)):
    columns.extend([
        f'Typ miejsca startu ({i})',
        f'Typ miejsca zakończenia ({i})',
    ])

columns

100%|██████████| 27/27 [00:00<?, ?it/s]


['ID odpowiedzi',
 'Typ miejsca startu (1)',
 'Typ miejsca zakończenia (1)',
 'Typ miejsca startu (2)',
 'Typ miejsca zakończenia (2)',
 'Typ miejsca startu (3)',
 'Typ miejsca zakończenia (3)',
 'Typ miejsca startu (4)',
 'Typ miejsca zakończenia (4)',
 'Typ miejsca startu (5)',
 'Typ miejsca zakończenia (5)',
 'Typ miejsca startu (6)',
 'Typ miejsca zakończenia (6)',
 'Typ miejsca startu (7)',
 'Typ miejsca zakończenia (7)',
 'Typ miejsca startu (8)',
 'Typ miejsca zakończenia (8)',
 'Typ miejsca startu (9)',
 'Typ miejsca zakończenia (9)',
 'Typ miejsca startu (10)',
 'Typ miejsca zakończenia (10)',
 'Typ miejsca startu (11)',
 'Typ miejsca zakończenia (11)',
 'Typ miejsca startu (12)',
 'Typ miejsca zakończenia (12)',
 'Typ miejsca startu (13)',
 'Typ miejsca zakończenia (13)',
 'Typ miejsca startu (14)',
 'Typ miejsca zakończenia (14)',
 'Typ miejsca startu (15)',
 'Typ miejsca zakończenia (15)',
 'Typ miejsca startu (16)',
 'Typ miejsca zakończenia (16)',
 'Typ miejsca startu (17

In [73]:
x = df[columns]

In [74]:
x.to_csv('typy_mijsc.csv')

In [66]:
import numpy 

In [67]:
y = x[~numpy.isnan(x)]

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''